In [7]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
#@title Run this cell first to set up the Python environment
#@markdown Setup will include installation and configuration of MNE Python and Matplotlib
from IPython.display import clear_output
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
#%pylab inline
!pip install mne                    #install the MNE software
import mne                          #get the MNE library
import os                           #get the os library
import urllib.request               #functions for retrieving data from URL
import pandas as pd
!pip install ipympl                 #not sure if this is needed
import ipympl                       #not sure if this is needed
import matplotlib.pyplot as plt                   
mne.set_log_level('WARNING')        #I believe this makes the MNE functions less verbose when they run
#from matplotlib import interactive  #not sure if needed
#interactive(True)                   #not sure if needed
%matplotlib inline
clear_output() 

In [9]:
#@title Load your data
#@markdown Copy your own links below, or use default value
EEGlab_set_file = 'https://drive.google.com/uc?export=download&id=1KsohSd2ODzA4oxXhyqIV-3x0M1WOL0wf'  #@param {type: "string"}
EEGlab_fdt_file = 'https://drive.google.com/uc?export=download&id=1cWmhfSAHD85ideF6Jyhd8bseKwB92wgN'  #@param {type: "string"}

urllib.request.urlretrieve(EEGlab_set_file, 'eeglabdata.set')
urllib.request.urlretrieve(EEGlab_fdt_file, 'eeglabdata.fdt')

#os.listdir()
raw=mne.io.read_raw_eeglab("eeglabdata.set", eog=(), preload=False, uint16_codec=None, verbose=True)
events_from_annot, event_dict = mne.events_from_annotations(raw)
#select channel types
raw.pick_types(meg=False, eeg=True, eog=False)

Reading /content/eeglabdata.fdt


<ipython-input-9-e35bcd7ede61>:10: RuntimeWarning: Data file name in EEG.data (MNItest.fdt) is incorrect, the file name must have changed on disk, using the correct file name (eeglabdata.fdt).
  raw=mne.io.read_raw_eeglab("eeglabdata.set", eog=(), preload=False, uint16_codec=None, verbose=True)


<RawEEGLAB | eeglabdata.fdt, 65 x 186875 (1495.0 s), ~92 kB, data not loaded>

In [10]:
#@title Re-reference
#@markdown skip this if not re-referencing data
reftype = "average"  #@param ['average', 'none']

raw.set_eeg_reference(reftype, projection=True)  # set EEG average reference

<RawEEGLAB | eeglabdata.fdt, 65 x 186875 (1495.0 s), ~94 kB, data not loaded>

In [11]:
#JUST HAVE TO FIX SOME CHANNEL LABELS THAT DON'T MATCH THE MNE TEMPLATE
#montage.ch_names
ChanMap={'FP1':'Fp1', 'FPZ':'Fpz', 'FP2':'Fp2', 'FZ':'Fz', 'FCZ':'FCz', 'CZ':'Cz', 'CPZ':'CPz', 'PZ':'Pz', 'AFZ':'AFz', 'POZ':'POz', 'OZ':'Oz'}
raw.rename_channels(ChanMap)

#ONCE THE CHANNELS ARE NAMED CORRECTLY WE CAN ADD 3D LOCATIONS BY SETTING THE MONTAGE
raw_1020 = raw.copy().set_montage('standard_1005')
del raw #save disk space

In [23]:
#@title Default title text
#import mne
import ipywidgets as widgets
from IPython.display import display
from google.colab import output
output.enable_custom_widget_manager()
# Load the sample EEG dataset
#data_path = mne.datasets.sample.data_path()
#raw_file = str(data_path) + '/MEG/sample/sample_audvis_raw.fif'
#raw = mne.io.read_raw_fif(raw_file)

# Create a widget for selecting the time range
time_range_slider = widgets.FloatRangeSlider(
    value=[0.0, 2.0], min=0.0, max=raw_1020.times.max(), step=1,
    description='Time Range', continuous_update=False
)

# Create a widget for selecting the window size
window_size_slider = widgets.FloatSlider(
    value=10.0, min=1, max=50.0, step=1,
    description='Step Size', continuous_update=False
)

# Create buttons for moving the time range
previous_button = widgets.Button(description='Previous')
next_button = widgets.Button(description='Next')

# Define a function to handle button clicks
def handle_button_clicks(button):
    current_time_range = time_range_slider.value
    window_size = window_size_slider.value
    step_size = window_size
    
    if button.description == 'Previous':
        new_start = current_time_range[0] - step_size
        new_end = current_time_range[1] - step_size
    elif button.description == 'Next':
        new_start = current_time_range[0] + step_size
        new_end = current_time_range[1] + step_size
    
    time_range_slider.value = [new_start, new_end]

# Register the button click handlers
previous_button.on_click(handle_button_clicks)
next_button.on_click(handle_button_clicks)

# Define a function to update the plot based on the widget values
def update_plot(time_range, window_size):
    fig = raw_1020.plot(start=time_range[0], duration=time_range[1] - time_range[0],
                   show_scrollbars=False, block=True)
    fig.canvas.manager.set_window_title('EEG Data')
    print(f'Time Range: {time_range}, Window Size: {window_size}')

# Create an output widget to display the plot
EEGoutput = widgets.Output()


# Define a function to handle widget value changes
def handle_value_change(change):
    EEGoutput.clear_output(wait=True)
    with EEGoutput:
        update_plot(time_range_slider.value, window_size_slider.value)

# Register the value change handlers
time_range_slider.observe(handle_value_change, 'value')
window_size_slider.observe(handle_value_change, 'value')

# Display the widgets, buttons, and output
display(time_range_slider)
display(window_size_slider)
display(previous_button)
display(next_button)
display(EEGoutput)
with EEGoutput:
  update_plot(time_range_slider.value, window_size_slider.value)

FloatRangeSlider(value=(0.0, 2.0), continuous_update=False, description='Time Range', max=1494.992, step=1.0)

FloatSlider(value=10.0, continuous_update=False, description='Step Size', max=50.0, min=1.0, step=1.0)

Button(description='Previous', style=ButtonStyle())

Button(description='Next', style=ButtonStyle())

Output()

In [ ]:
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))